In [67]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

In [86]:
df = pd.read_csv(r'train_rev.csv')

In [87]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'review', 'recommendation', 'game_name'], dtype='object')

In [88]:
X = df['review'].values

y2 = df['game_name'].values

In [71]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['recommendation'] = label_encoder.fit_transform(df['recommendation'])  



In [89]:
unique_values = df['recommendation'].unique()
print(unique_values)


[1 0]


In [90]:
y1 = df['recommendation'].values

In [91]:
df['length'] = df['review'].apply(lambda x: len(x.split()))

# Tentukan nilai persentil ke-90 dari panjang review
percentile_90 = np.percentile(df['length'], 90)
print(f'Panjang persentil ke-90: {percentile_90}')

# Gunakan nilai ini sebagai maxlen
maxlen = int(percentile_90)
print(f'Panjang maxlen yang digunakan: {maxlen}')

Panjang persentil ke-90: 211.0
Panjang maxlen yang digunakan: 211


In [92]:
# Membuat Tokenizer
tokenizer = Tokenizer(num_words=2000, oov_token="<OOV>")
tokenizer.fit_on_texts(X)

# Mengonversi teks menjadi urutan
sequences = tokenizer.texts_to_sequences(X)

In [93]:
sequences = tokenizer.texts_to_sequences(X)
word_index = tokenizer.word_index

In [94]:
maxlen=211
padded_sequences = pad_sequences(sequences, maxlen=maxlen, padding='post', truncating='post')

In [95]:
unique_values = df['recommendation'].unique()
print(unique_values)

[1 0]


In [96]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train1, y_test1= train_test_split(padded_sequences, y1, test_size=0.2, random_state=42)

In [97]:
s=2000
model = Sequential()
model.add(Embedding(input_dim=s, output_dim=128, input_length=maxlen))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(32))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  # Untuk binary classification

# 6. Mengompilasi model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 7. Melatih model
model.fit(X_train, y_train1, epochs=10, batch_size=32, validation_data=(X_test, y_test1))

# 8. Evaluasi model
loss, accuracy = model.evaluate(X_test, y_test1)
print(f'Test accuracy: {accuracy:.2f}')

Epoch 1/10


c:\Users\USER\Documents\projek\Steam_pred\bakso2\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


466/466 ━━━━━━━━━━━━━━━━━━━━ 78s 160ms/step - accuracy: 0.5175 - loss: 0.6904 - val_accuracy: 0.5385 - val_loss: 0.6833
Epoch 2/10
466/466 ━━━━━━━━━━━━━━━━━━━━ 86s 184ms/step - accuracy: 0.5260 - loss: 0.6805 - val_accuracy: 0.5396 - val_loss: 0.6814
Epoch 3/10
466/466 ━━━━━━━━━━━━━━━━━━━━ 93s 199ms/step - accuracy: 0.5446 - loss: 0.6634 - val_accuracy: 0.5380 - val_loss: 0.6867
Epoch 4/10
466/466 ━━━━━━━━━━━━━━━━━━━━ 99s 212ms/step - accuracy: 0.5576 - loss: 0.6528 - val_accuracy: 0.5490 - val_loss: 0.6986
Epoch 5/10
466/466 ━━━━━━━━━━━━━━━━━━━━ 98s 210ms/step - accuracy: 0.5728 - loss: 0.6442 - val_accuracy: 0.6768 - val_loss: 0.6505
Epoch 6/10
466/466 ━━━━━━━━━━━━━━━━━━━━ 97s 207ms/step - accuracy: 0.7043 - loss: 0.5717 - val_accuracy: 0.6719 - val_loss: 0.6619
Epoch 7/10
466/466 ━━━━━━━━━━━━━━━━━━━━ 105s 225ms/step - accuracy: 0.7058 - loss: 0.5644 - val_accuracy: 0.6244 - val_loss: 0.6328
Epoch 8/10
466/466 ━━━━━━━━━━━━━━━━━━━━ 119s 256ms/step - accuracy: 0.7479 - loss: 0.5168 - v

In [98]:
y_pred = model.predict(X_test)

117/117 ━━━━━━━━━━━━━━━━━━━━ 7s 53ms/step


In [99]:
y_pred_classes = (y_pred > 0.5).astype(int).flatten()

In [100]:
from sklearn.metrics import classification_report, confusion_matrix
print("Classification Report:\n", classification_report(y_test1, y_pred_classes))

Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.79      0.80      1871
           1       0.80      0.82      0.81      1854

    accuracy                           0.81      3725
   macro avg       0.81      0.81      0.81      3725
weighted avg       0.81      0.81      0.81      3725



In [120]:
x_rl = ["This game is very boring, it's better to save your money, friend"]
x_rl_tokenized = tokenizer.texts_to_sequences(x_rl)

x_rl_padded = pad_sequences(x_rl_tokenized, maxlen=maxlen)
yrl = model.predict(x_rl_padded)
yrl_class = (yrl > 0.5).astype(int)
if yrl_class == 1:
    print('Recommended')
elif yrl_class == 0:
    print('Not Recomeded')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Not Recomeded


In [119]:
model.save('NLP_Sentiment.h5')